In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('../input/111111/y_train.csv')
y_dic=df['y'].to_numpy()
y_dic

In [ ]:
# load data

import glob 
from PIL import Image

TRAIN_DIR="../input/111111/train_fig/train_fig"
TEST_DIR="../input/111111/test_fig/test_fig"

def load_data():
    file_list = glob.glob(TRAIN_DIR + "/*.jpg")
    x = []
    y = []
    
    for fname in file_list:
        with Image.open(fname) as img:
            np_img = np.array(img)
        label = int(fname[-8:-4]) 

        x.append(np_img)
        y.append(y_dic[label])
        
    x, y = np.array(x), np.array(y)
    x = np.expand_dims(x, -1)
    x = x / 255.
    return x, y
    
x_train,y_train = load_data()


In [ ]:
import matplotlib.pyplot as plt
plt.subplot(141)
plt.imshow(x_train[0].squeeze(2))
plt.subplot(142)
plt.imshow(x_train[1].squeeze(2))
plt.subplot(143)
plt.imshow(x_train[2].squeeze(2))
plt.subplot(144)
plt.imshow(x_train[3].squeeze(2))

In [ ]:
y_train[0], y_train[1], y_train[2], y_train[3]

In [ ]:
def load_test():
    file_list = glob.glob(TEST_DIR + "/*.jpg")
    x = [] 
    y = []
    for fname in file_list:
        with Image.open(fname) as img:
            np_img = np.array(img)
        label = int(fname[-8:-4]) 

        x.append(np_img)
        y.append(label)
        
    x, y = np.array(x), np.array(y)
    x = np.expand_dims(x, -1)
    x = x / 255.
    return x, y
    
x_test, y_test = load_test()

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
model = tf.keras.models.Sequential()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten

In [ ]:
model = Sequential([
    Conv2D(6, 5, activation='relu', input_shape=x_train.shape[1:]),
    MaxPool2D(2),
    Conv2D(16, 5, activation='relu',),
    MaxPool2D(2),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(15, activation='softmax')])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

# configure the model for training
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['acc'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=4, verbose=2)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
test_loss,test_accuracy = model.evaluate(x_test,y_test)

In [57]:
pred = model.predict_classes(x_test)
output = pd.DataFrame(pred,columns=["y"])
output = output.set_index(y_test)
output.index.name = 'id'

output
output.to_csv('submittion.csv')

NameError: name 'id_test' is not defined